In [2]:
import pandas as pd
pd.__version__

'2.2.2'

In [30]:
taxi_data_1 = pd.read_parquet('data/yellow_tripdata_2024-01.parquet')
taxi_data_2 = pd.read_parquet('data/yellow_tripdata_2024-02.parquet')

In [31]:
print('Number of columns in the Jan data {}'.format(len(taxi_data_1.columns)))

Number of columns in the Jan data 19


### Homework Question 1: Number of Columns

In [32]:
"There are [{}] columns in the yellow trip data for the month of January.".format(len(taxi_data_1.columns))

'There are [19] columns in the yellow trip data for the month of January.'

In [6]:
'''
Counting the non-NA cells for each column or row
'''
print("Total number of rows {}".format(len(taxi_data_1)))
print()
print('Number of non zero values in each column')
print(taxi_data_1.count())

Total number of rows 2964624

Number of non zero values in each column
VendorID                 2964624
tpep_pickup_datetime     2964624
tpep_dropoff_datetime    2964624
passenger_count          2824462
trip_distance            2964624
RatecodeID               2824462
store_and_fwd_flag       2824462
PULocationID             2964624
DOLocationID             2964624
payment_type             2964624
fare_amount              2964624
extra                    2964624
mta_tax                  2964624
tip_amount               2964624
tolls_amount             2964624
improvement_surcharge    2964624
total_amount             2964624
congestion_surcharge     2824462
Airport_fee              2824462
dtype: int64


### Homework Question 2: Computing Duration

In [7]:
# Convert the columns to datetime if they are not already
taxi_data_1['tpep_pickup_datetime'] = pd.to_datetime(taxi_data_1['tpep_pickup_datetime'])
taxi_data_1['tpep_dropoff_datetime'] = pd.to_datetime(taxi_data_1['tpep_dropoff_datetime'])

# Calculate the duration in minutes
taxi_data_1['duration_in_minutes'] = (taxi_data_1['tpep_dropoff_datetime'] - taxi_data_1['tpep_pickup_datetime']).dt.total_seconds() / 60

# Display the DataFrame with the new duration column
print(taxi_data_1[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'duration_in_minutes']].head())

  tpep_pickup_datetime tpep_dropoff_datetime  duration_in_minutes
0  2024-01-01 00:57:55   2024-01-01 01:17:43            19.800000
1  2024-01-01 00:03:00   2024-01-01 00:09:36             6.600000
2  2024-01-01 00:17:06   2024-01-01 00:35:01            17.916667
3  2024-01-01 00:36:38   2024-01-01 00:44:56             8.300000
4  2024-01-01 00:46:51   2024-01-01 00:52:57             6.100000


In [8]:
taxi_data_1.duration_in_minutes.describe()["std"]

34.851053592212814

### Homework Question 3: Dropping Outliers

In [9]:
taxi_data_1.duration_in_minutes.describe()

count    2.964624e+06
mean     1.561295e+01
std      3.485105e+01
min     -1.356667e+01
25%      7.150000e+00
50%      1.163333e+01
75%      1.868333e+01
max      9.455400e+03
Name: duration_in_minutes, dtype: float64

In [10]:
# Remove outliers (keep records where the duration was between 1 and 60 minutes inclusive)
taxi_filtered = taxi_data_1[(taxi_data_1['duration_in_minutes'] >= 1) & (taxi_data_1['duration_in_minutes'] <= 60)]

In [11]:
print("The number of records left after filtering {}%".format(round(len(taxi_filtered) / len(taxi_data_1) * 100), 4))

The number of records left after filtering 98%


### Homework Question 4: One Hot Encoding

In [12]:
taxi_data_1.PULocationID.dtype

dtype('int32')

In [13]:
print(len(taxi_data_1.PULocationID.unique()))
print(len(taxi_data_1.DOLocationID.unique()))

260
261


In order to do one-hot encoding on the pickup/dropoff location ID, we need to first convert the variable into a string and then proceed with the next steps

In [14]:
from sklearn.feature_extraction import DictVectorizer

def data_transformations(df: pd.DataFrame, datatype: str, vectorizer: DictVectorizer):
    # Convert pickup and dropoff location IDs to strings
    df['PULocationID'] = df['PULocationID'].astype(str)
    df['DOLocationID'] = df['DOLocationID'].astype(str)

    # Create a list of dictionaries from the DataFrame
    records = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
    
    
    if datatype == 'train':
        X = vectorizer.fit_transform(records)
    else:
        X = vectorizer.transform(records)
        
    # Print the feature names and the feature matrix
    print('The dimensionality of the DictVectorizer for the {} data is {}'.format(datatype, X.shape))
    return X    

In [16]:
# Fit a dictionary vectorizer
vectorizer = DictVectorizer(sparse=True)
X = data_transformations(taxi_filtered, 'train', vectorizer)

/tmp/ipykernel_33532/4128944378.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PULocationID'] = df['PULocationID'].astype(str)
/tmp/ipykernel_33532/4128944378.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOLocationID'] = df['DOLocationID'].astype(str)


The dimensionality of the DictVectorizer for the train data is (2898906, 518)


### Homework Question 5: Training a Model. RMSE on train

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [18]:
# label is obviously the duration of the trip in minutes (we created this in the previous steps)
y = taxi_filtered['duration_in_minutes'].values

In [19]:
print(y.shape, X.shape)

(2898906,) (2898906, 518)


In [20]:
# Train a simple linear regression model on the data 
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [21]:
# Predict on the training data
y_pred = model.predict(X)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))

print("RMSE on training data:", rmse)

RMSE on training data: 7.946174674221988


### Homework Question 6: Evaluating the model (Feb). RMSE on validation

In [22]:
# Applying the same data transformations on the 'taxi dataset from feb 2023'

# Convert the columns to datetime if they are not already
taxi_data_2['tpep_pickup_datetime'] = pd.to_datetime(taxi_data_2['tpep_pickup_datetime'])
taxi_data_2['tpep_dropoff_datetime'] = pd.to_datetime(taxi_data_2['tpep_dropoff_datetime'])

# Calculate the duration in minutes
taxi_data_2['duration_in_minutes'] = (taxi_data_2['tpep_dropoff_datetime'] - taxi_data_2['tpep_pickup_datetime']).dt.total_seconds() / 60

In [23]:
taxi_data_2.duration_in_minutes.describe()

count    3.007526e+06
mean     1.598243e+01
std      3.428955e+01
min     -5.206667e+01
25%      7.366667e+00
50%      1.200000e+01
75%      1.920000e+01
max      5.671983e+03
Name: duration_in_minutes, dtype: float64

In [24]:
# Remove outliers (keep records where the duration was between 1 and 60 minutes inclusive)
taxi_2_filtered = taxi_data_2[(taxi_data_2['duration_in_minutes'] >= 1) & (taxi_data_2['duration_in_minutes'] <= 60)]

In [25]:
print("The number of records left after filtering {}%".format(round(len(taxi_2_filtered) / len(taxi_data_2) * 100), 4))

The number of records left after filtering 98%


In [26]:
X_test = data_transformations(taxi_2_filtered, 'test', vectorizer)

/tmp/ipykernel_33532/4128944378.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PULocationID'] = df['PULocationID'].astype(str)
/tmp/ipykernel_33532/4128944378.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOLocationID'] = df['DOLocationID'].astype(str)


The dimensionality of the DictVectorizer for the test data is (2938060, 518)


In [27]:
y_test = taxi_2_filtered.duration_in_minutes
y_test.shape

(2938060,)

In [28]:
y_pred = model.predict(X_test)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("RMSE on validation (feb) data:", rmse)

RMSE on validation (feb) data: 8.123392575033122
